In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#data dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [0]:
#Aunthenticate

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Training Link : https://drive.google.com/open?id=1NWrf1adDUBbA5MM3q-gqr0bZu1SrtILX
# Testing Link : https://drive.google.com/open?id=1RHIIZ34VGN5BEha3_OMSq4CmVVxvY-Fm

train_downloaded = drive.CreateFile({'id':'1NWrf1adDUBbA5MM3q-gqr0bZu1SrtILX'})
train_downloaded.GetContentFile('clean_tweet_training.csv')
test_downloaded = drive.CreateFile({'id':'1RHIIZ34VGN5BEha3_OMSq4CmVVxvY-Fm'})
test_downloaded.GetContentFile('clean_tweet_testing.csv')

In [0]:
train_df = pd.read_csv('clean_tweet_training.csv')
test_df = pd.read_csv('clean_tweet_testing.csv')

In [0]:
# Pretrained Link : https://drive.google.com/a/mail.ugm.ac.id/uc?export=download&confirm=19S_&id=1-UJRiTvSnmyrK61CDiCFq5ql1q_mK5D-

pretrained_download = drive.CreateFile({'id':'1-UJRiTvSnmyrK61CDiCFq5ql1q_mK5D-'})
pretrained_download.GetContentFile('GoogleNews.bin')

In [0]:
train_df.head()

,index,text,target
0,0,awww bummer shoulda got david carr third day,0
1,1,upset update facebook texting might cry result...,0
2,2,dived many times ball managed save rest go bounds,0
3,3,whole body feels itchy like fire,0
4,4,behaving mad see,0


In [0]:
test_df.head()

,index,text,target
0,0,loooooooovvvvvveee kindle dx cool fantastic right,1.0
1,1,reading kindle love lee childs good read,1.0
2,2,ok first assesment kindle fucking rocks,1.0
3,3,love kindle mine months never looked back new ...,1.0
4,4,fair enough kindle think perfect,1.0


In [0]:
print('Processing text dataset')
from nltk.tokenize import WordPunctTokenizer
from collections import Counter
from string import punctuation, ascii_lowercase
import regex as re
from tqdm import tqdm

# replace urls
re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                    .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
re.MULTILINE|re.UNICODE)

# replace ips
re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")

# setup tokenizer
tokenizer = WordPunctTokenizer()
vocab = Counter()

def text_to_wordlist(text, lower=False):
    # replace URLs
    text = re_url.sub("URL", text)
    # replace IPs
    text = re_ip.sub("IPADDRESS", text)
    # Tokenize
    text = tokenizer.tokenize(text)
    # optional: lower case
    if lower:
        text = [t.lower() for t in text]
        # Return a list of words
        vocab.update(text)
    return text

def process_comments(list_sentences, lower=False):
    comments = []
    for text in tqdm(list_sentences):
        txt = text_to_wordlist(text, lower=lower)
        comments.append(txt)
    return comments

list_sentences_train = list(train_df["text"].fillna("NAN_WORD").values)
list_sentences_test = list(test_df["text"].fillna("NAN_WORD").values)
comments = process_comments(list_sentences_train + list_sentences_test, lower=True)

Processing text dataset


100%|██████████| 1600498/1600498 [00:19<00:00, 82147.40it/s]


In [0]:
print("The vocabulary contains {} unique tokens".format(len(vocab)))

The vocabulary contains 273627 unique tokens


In [0]:
import gensim
from gensim.models import Word2Vec

In [0]:
import gensim
from gensim.models import KeyedVectors

# Load pre-trained Word2Vec model.
wv_from_bin = KeyedVectors.load_word2vec_format("GoogleNews.bin", binary=True)  # C bin format

In [0]:
word_vectors = wv_from_bin.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [0]:
# wv_from_bin.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

In [0]:
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 200

In [0]:
from keras.preprocessing.sequence import pad_sequences

word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(MAX_NB_WORDS))}
sequences = [[word_index.get(t, 0) for t in comment]
             for comment in comments[:len(list_sentences_train)]]
test_sequences = [[word_index.get(t, 0)  for t in comment] 
                  for comment in comments[len(list_sentences_train):]]

# pad
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, 
                     padding="pre", truncating="post")
list_classes = ["target"]
y = train_df[list_classes].values
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",
                          truncating="post")
print('Shape of test_data tensor:', test_data.shape)

Using TensorFlow backend.


Shape of data tensor: (1600000, 200)
Shape of label tensor: (1600000, 1)
Shape of test_data tensor: (498, 200)


In [0]:
WV_DIM = 100
nb_words = min(MAX_NB_WORDS, len(word_vectors.vocab))
# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(nb_words, WV_DIM) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass      

In [0]:
from keras.layers import Dense, Input, Conv1D, MaxPooling1D, BatchNormalization, LSTM, Dense, Dropout, Embedding, SpatialDropout1D, CuDNNLSTM, Bidirectional
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

In [0]:
wv_layer = Embedding(nb_words,
                     WV_DIM,
                     mask_zero=False,
                     weights=[wv_matrix],
                     input_length=MAX_SEQUENCE_LENGTH,
                     trainable=False)

# Inputs
comment_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = wv_layer(comment_input)

# Embedded
embedded_sequences = SpatialDropout1D(0.2)(embedded_sequences)

#Convolutional1D
x = Conv1D(filters=3, kernel_size=64, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, activation='relu', use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(embedded_sequences)

#MaxPoolingLayer1D
x = MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')(x)

#DropoutLayers
x = Dropout(rate=0, noise_shape=None, seed=None)(x)

#BatchNormalization
x = BatchNormalization()(x)

#LSTM
x = LSTM(units=128, activation='relu', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initiali0zer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0, implementation=1, return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)(x)

preds = Dense(1, activation='relu')(x)

# build the model
model = Model(inputs=[comment_input], outputs=preds)
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001, clipnorm=.25, beta_1=0.7, beta_2=0.99),
              metrics=['accuracy'])

In [0]:
hist = model.fit([data], y, validation_split=0.1, epochs=8, batch_size=256, shuffle=True)

In [0]:
model.summary()

In [0]:
history = pd.DataFrame(hist.history)
plt.figure(figsize=(12,12));
plt.plot(history["loss"]);
plt.plot(history["val_loss"]);
plt.title("Loss with pretrained word vectors");
plt.show();

In [0]:
acc = history.iloc[0:6,3]
loss = history.iloc[0:6,2]
history

In [0]:
plt.plot(acc, color='g');
# plt.plot(loss, color='blue');
plt.title("Accuray Function");
plt.show();
# plt.plot(acc, color='g');
plt.plot(loss, color='blue');
plt.title("Loss Function");
plt.show();

NameError: ignored